## Download SCAMPR through Amazon S3

In [1]:
import xarray as xr
import numpy as np
# import rioxarray
import boto3
from botocore import UNSIGNED
from datetime import timezone, datetime, timedelta
from pandas import date_range

In [ ]:
datelist = date_range(start='2025-02-11', end='2025-08-28', freq='h')

for dtime in datelist:
    print("Processing:", dtime)
    # Inisialisasi client S3 tanpa autentikasi (karena bucket public)
    s3 = boto3.client('s3', config=boto3.session.Config(signature_version=UNSIGNED))
    
    bucket_name = "noaa-enterprise-rainrate-pds"
    prefix = f"BLEND/RainRate-Blend-INST/{dtime:%Y}/{dtime:%m}/{dtime:%d}/{dtime:%H}/"
    
    # Ambil list objek
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    objects = response.get("Contents", [])
    filtered = [obj for obj in objects if "GLB-5" in obj["Key"]]
    for latest_object in filtered:
        local_dir_nc = "G:/SCAMPR/nc"
        local_filename_nc = f"{local_dir_nc}/{latest_object['Key'].split('/')[-1]}"
        s3.download_file(bucket_name, latest_object["Key"], local_filename_nc)

## Clip and Convert to GeoTIFF

In [ ]:
import os
import rioxarray

In [ ]:
nc_dir = "G:/SCAMPR/nc"
filelist = os.listdir(nc_dir)
domain = (105.0, -8.0, 108.9, -5.7)  # (minx, miny, maxx, maxy)

In [ ]:
for file in filelist:
    ds = xr.open_dataset(os.path.join(nc_dir,file), engine='netcdf4')
    
    # Original dimension
    nrows = ds.sizes["Rows"]   # atau "row"
    ncols = ds.sizes["Columns"] # atau "column"
    
    # metadata
    lat_min = float(ds.geospatial_lat_min)
    lat_max = float(ds.geospatial_lat_max)
    lon_min = float(ds.geospatial_lon_min)
    lon_max = float(ds.geospatial_lon_max)
    
    # generate grids coordinates
    lats = np.linspace(lat_max, lat_min, nrows)   # north → south
    lons = np.linspace(lon_min, lon_max, ncols)
    
    # Assign koordinat
    ds = ds.assign_coords(
        lat=("Rows", lats),
        lon=("Columns", lons)
    )
    
    # Ganti dimensi: row→lat, column→lon
    ds = ds.swap_dims({"Rows": "lat", "Columns": "lon"})
    
    # Tambahkan atribut CF
    ds["lat"].attrs = {
        "standard_name": "latitude",
        "long_name": "latitude",
        "units": "degrees_north"
    }
    ds["lon"].attrs = {
        "standard_name": "longitude",
        "long_name": "longitude",
        "units": "degrees_east"
    }
    
    sliced = ds.sel(lat=slice(domain[3],domain[1]), lon=slice(domain[0],domain[2]))
    sliced = sliced['RRQPE'].squeeze()
    sliced = sliced.rio.write_crs("EPSG:4326")
    sliced = sliced.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    
    tif_dir = "G:/SCAMPR/tif"
    time_indicator = file.split('_')[3]
    tif_file = f"{tif_dir}/rrqpe_jawa_{time_indicator}.tif"
    sliced.rio.to_raster(tif_file, compression='LZW', dtype='float32')
    print("Saved:", tif_file)
    ds.close()

Modify the script as needed to automate the data fetching and clipping for operational purpose